In [3]:
import boto3
from configparser import ConfigParser
import json
import pandas as pd
import io

In [4]:
# Carregar as configurações do arquivo ini
config = ConfigParser()
config.read("../config/config.ini")

['../config/config.ini']

In [5]:
# Configurações para conexão com o MinIO/S3
endpoint = config.get("MinIO", "endpoint")
access_key = config.get("MinIO", "access_key")
secret_key = config.get("MinIO", "secret_key")
bucket_raw = config.get("Bucket", "bucket_raw")
bucket_context = config.get("Bucket", "bucket_context")
prefix_coordenadas_json = config.get("Bucket", "prefix_coordenadas_json") # Prefixo do arquivo CSV na raw
prefix_coordenadas_csv = config.get("Bucket", "prefix_coordenadas_csv") # Prefixo do arquivo somente com as informações de saúde que vai para a context

# Nome do arquivo a ser lido
source_filename = config.get("FILE", "coordenadas_municipios_json")
target_filename = config.get("FILE", "coordenadas_municipios_csv")

In [6]:
print("endpoint:", endpoint)
print("access_key:", access_key)
print("secret_key:", secret_key)
print("bucket_raw:", bucket_raw)
print("bucket_context:", bucket_context)
print("prefix_coordenadas_json:", prefix_coordenadas_json)
print("prefix_coordenadas_csv:", prefix_coordenadas_csv)
print("source_filename:", source_filename)
print("target_filename:", target_filename)

endpoint: http://minio:9000
access_key: aulafia
secret_key: aulafia@123
bucket_raw: raw
bucket_context: context
prefix_coordenadas_json: coordenadas_json/
prefix_coordenadas_csv: coordenadas_csv/
source_filename: Coordenadas.json
target_filename: Coordenadas.csv


In [7]:
# Inicializar o cliente boto3 para S3
minio_client = boto3.client("s3", 
                            endpoint_url=endpoint,
                            aws_access_key_id=access_key,
                            aws_secret_access_key=secret_key
)

In [9]:
# Obter o objeto do arquivo do bucket de origem
response = minio_client.get_object(Bucket=bucket_raw,
                                   Key=prefix_coordenadas_json + 
                                   source_filename)
json_content = response['Body'].read()

In [10]:
# Decodificar o conteúdo JSON
decoded_json = json.loads(json_content)

In [11]:
# Converter o JSON para um DataFrame do pandas
data_frame = pd.json_normalize(decoded_json)

In [12]:
# Criar um fluxo de gravação em buffer para o arquivo CSV
output_buffer = io.StringIO()

In [13]:
# Salvar o DataFrame como um arquivo CSV no buffer
data_frame.to_csv(output_buffer, index=False, encoding='utf-8')

In [14]:
# Obter o conteúdo do buffer como bytes codificados em UTF-8
encoded_csv_content = output_buffer.getvalue().encode('utf-8')

In [15]:
# Enviar o arquivo CSV codificado para o bucket de destino
minio_client.put_object(Bucket=bucket_context,
                        Key=target_filename,
                        Body=encoded_csv_content)

{'ResponseMetadata': {'RequestId': '177F60666F2E236A',
  'HostId': 'dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'accept-ranges': 'bytes',
   'content-length': '0',
   'etag': '"92055ff5e21661003b819ec7fac8a438"',
   'server': 'MinIO',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'vary': 'Origin, Accept-Encoding',
   'x-amz-id-2': 'dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8',
   'x-amz-request-id': '177F60666F2E236A',
   'x-content-type-options': 'nosniff',
   'x-xss-protection': '1; mode=block',
   'date': 'Sun, 27 Aug 2023 23:12:58 GMT'},
  'RetryAttempts': 0},
 'ETag': '"92055ff5e21661003b819ec7fac8a438"'}